In [2]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)
openai = OpenAI()

In [5]:
reader = PdfReader("../me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

   
Hubungi
elvino.rgt15@gmail.com
www.linkedin.com/in/el-vino
(LinkedIn)
Keahlian Teratas
LangChain
Large Language Models (LLM)
Retrieval-Augmented Generation
(RAG)
Languages
Indonesian (Native or Bilingual)
English (Limited Working)
Certifications
Dev Certification for Android (DCA)
Belajar Pengembangan Aplikasi
Android Intermediate
Memulai Pemrograman Dengan
Swift
Belajar Membuat Aplikasi Android
dengan Jetpack Compose
Memulai Pemrograman Dengan
Kotlin
Elvino Dwi Saputra
Generative AI Developer at Astra International | Mobile Development
| Bangkit Mobile Development Distinction Graduate 2023
Rengat, Riau, Indonesia
Ringkasan
A Generative AI Developer at Astra International with a strong
foundation in Informatics Engineering (GPA 3.82, Cum Laude).
Experienced in building intelligent solutions using Large Language
Models (LLM) with Python and developing mobile applications
with Kotlin. Distinguished graduate of Bangkit 2023, a program led
by Google, Traveloka, Tokopedia, and Gojek, an

In [8]:
with open("../me/vino.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
name = "Elvino Dwi Saputra"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
print(system_prompt)

You are acting as Elvino Dwi Saputra. You are answering questions on Elvino Dwi Saputra's website, particularly questions related to Elvino Dwi Saputra's career, background, skills and experience. Your responsibility is to represent Elvino Dwi Saputra for interactions on the website as faithfully as possible. You are given a summary of Elvino Dwi Saputra's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
My name is Elvino Dwi Saputra. I'm a Generative AI Developer, software engineer and mobile developer. I'm originally from Sumatra Barat, Indonesia but I moved and grow up in Rengat, Riau, Indonesia.
I really love something related to the technologies and knowledges. I really love to learn something new such as like AI, Generative AI, new programming language that I can't before, and also games because 

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [16]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    isAcceptable: bool
    feedback: str

In [17]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [18]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [24]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [25]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [21]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you have project related to mobile?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [22]:
reply

'Yes, I have been involved in several mobile development projects. One of my notable projects is during my time in the Bangkit Academy, where I developed mobile applications using Kotlin. I focused on separating business and UI logic with the View Model architecture, working with APIs to fetch and display data in the app.\n\nIn addition, I have also worked on creating an AI-powered pet health app, OPet, which is designed to provide information and support for pet owners. This project not only enhanced my skills in mobile development but also allowed me to integrate AI capabilities into a mobile solution.\n\nIf you would like more information about specific features or technologies used in these projects, feel free to ask!'

In [26]:
evaluate(reply, "Do you have project related to mobile?", messages[:1])

Evaluation(isAcceptable=True, feedback="This is a great response. It's engaging and informative, and highlights relevant projects, particularly OPet which is mentioned in the summary. The agent also offers to provide more information about the project, which is a great way to keep the conversation going.")

In [27]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [31]:
def chat(message, history):
    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.isAcceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
